In [1]:
import numpy as np
import scipy
import scipy.ndimage as nd
from PIL import Image
import math

In [2]:
# read in 3 images as numpy arrays
img1 = Image.open('img01.jpg')
img1 = np.asarray(img1, dtype=np.int64)
img1 = np.transpose(img1, (1,0,2))

img2 = Image.open('img02.jpg')
img2 = np.asarray(img2, dtype=np.int64)
img2 = np.transpose(img2, (1,0,2))

img3 = Image.open('img03.jpg')
img3 = np.asarray(img3, dtype=np.int64)
img3 = np.transpose(img3, (1,0,2))

In [4]:
# Helper function which declares an array that will keep track of the probabilities of an pixel belonging to 
# a particular cluster.
def gen_temp_arrays(arr, num):
    arr1 = np.zeros((len(arr), len(arr[0]), num))
    return arr1

In [5]:
def find_wij(pixel, means, probs, cur_mean, num_means, temp_probs, i, j):
    '''
    Computes the W_ij term (essentially the probabilities for a pixel to belong to a particular blob).
    @param pixel: The current pixel for which we are computing the probabilities (the X_i).
    @param means: The current array of means for the j clusters.
    @param probs: The current array of proportions for the j clusters.
    @param cur_mean: The current mean index we are computing for (The u_j).
    @param num_means: The total number of means, which is equivilant to the total number of clusters.
    '''
    
    # First we need to compute the values of the values in the exponent.
    val = np.zeros((num_means, ))
    
    # For each of the clusters, we compute the inner value of the exponential.
    for i in range(num_means):
        val[i] = (-0.5*((pixel - means[i]).T @ ((pixel) -means[i])))

    # Then shift each of the values by subtracting from the largest element in the list.
    # This corrects for potential underflow or overflow issues.
    val -= val.max(axis = 0)
    
    # Compute the numerator which is the exponential of the gaussian expression with the cur_mean multiplied by 
    # the probability of the current blob being expressed by the dataset.
    numer = np.exp(val[cur_mean])*probs[cur_mean]
    
    # Now take the sum of the exponentials of the gaussian expression with each of the cluster means multiplied by
    # the probability of the "j_th" blob being expressed by the dataset.
    denom = 0
    for i in range(num_means):
        denom += np.exp(val[i])*probs[i]
    
    # Return the divison of each of these two numbers, which roughly expresses the probability of this pixel belonging to
    # the blob.
    wij = numer / denom
    temp_probs[i][j][cur_mean] = wij
    
    return numer / denom

In [6]:
def em_step(num_means, image):
    '''
    This function performs the iterative 'E' and 'M' step for the mixture of gaussians.
    @param num_means: The number of segments(clusters) to use.
    @param image: The image where the clustering will take place, represented as an numpy array.
    '''
    # Define a tolerance to which we know when to stop.
    eps = 2
    
    # Construct an initialize and populate an array to hold the proportions for each of the clusters.
    # In addition, initialize and populate an array to hold the means for each of the clusters.  
    init_pi = np.full((num_means,), 1./num_means)
    init_mean = np.zeros((num_means, 3))
    num_pixels = len(image) * len(image[0])
    # Set the initial mean vector to be randomly sampled pixels from the image.
    for m in range(num_means):
        rand_row = np.random.randint(0, len(image))
        rand_col = np.random.randint(0, len(image[0]))
        init_mean[m] = image[rand_row][rand_col]
    
    # Construct a vector that will hold the probabilities that a particular pixel belongs to a specific cluster.
    temp_probs = gen_temp_arrays(image, num_means)
    
    # Iterate over each cluster. For each cluster compute the probability that a pixel belongs to a cluster.
    # Use these probabilities to calculate a new approximation for both the mean and proportion of the cluster.
    for i in range(len(100)):
        for curMeanIndex in range(len(init_mean)):
            numerator = np.zeros((3, ))
            denomenator = 0
            for i in range(len(image)):
                for j in range(len(image[0])):
                    # Compute the probability that this pixel is a member of this cluster.
                    temp_pixel = image[i][j]
                    wij = find_wij(temp_pixel, init_mean, init_pi, curMeanIndex, num_means, temp_probs, i, j)

                    # Use this probability to compute the updated means and proportions for this dataset.
                    numerator += temp_pixel * wij
                    denomenator += wij


            # Update the value of the mean for the particular 'blob'.
            newMean = (numerator / denomenator).astype(np.int64)
            newPop = (denomenator / num_pixels)

            # If the difference is smaller than epsilon, then return and report the newMean, newPop, and temp_probs.
            if (np.linalg.norm(newMean - init_mean[curMeanIndex]) < eps):
                return newMean, newPop, temp_probs

            # Otherwise update the estimate of the mean and proportion of the particular cluster.
            init_mean[curMeanIndex] = newMean
            init_pi[curMeanIndex] = newPop
        
    return init_mean, init_pi, temp_probs        


[ 185.  231.  164.]
0.0448814132488


First, we will segment the first image into 10, 20, and 50 segments.

In [ ]:
clusters_1_10_mean, clusters_1_10_prop, clusters_1_10_probs = em_step(10, img1)

In [ ]:
# img1
info1, info2, info3 = gen_temp_arrays(img1)
